# Fraud Detection – Model Building and Training
## Objective
Build, train, and evaluate classification models to detect fraudulent
transactions using techniques appropriate for highly imbalanced data.
Models are compared using AUC-PR, F1-Score, and confusion matrices.
Now includes threshold optimization for improved performance.

### Load Feature-Engineered Data

In [ ]:
# 📦 Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sys
from pathlib import Path
import joblib
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from sklearn.metrics import precision_recall_curve, auc

# Add project root to sys.path
PROJECT_ROOT = Path("..").resolve()  # assuming notebook is in 'notebooks/'
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Import updated modules
from src.preprocessing import clean_fraud_data, build_preprocessor, separate_features_target
from src.data_split import stratified_split
from src.imbalance import apply_smote
from src.models import logistic_regression, random_forest, threshold_optimized_random_forest
from src.metrics import evaluate_model, evaluate_model_at_thresholds, find_optimal_threshold
from src.metrics import get_business_metrics
from src.cv import stratified_cv_with_threshold_opt, cv_threshold_optimized
from src.visualization import plot_confusion_matrix, plot_pr_curve, plot_threshold_analysis, plot_metric_tradeoff, save_figures

print("=" * 80)
print("FRAUD DETECTION MODELING WITH THRESHOLD OPTIMIZATION")
print("=" * 80)

# 1️⃣ Load Feature-Engineered Data
print("\n1. Loading feature-engineered data...")
df_raw = pd.read_csv("../data/processed/fraud_data_features.csv")
df = clean_fraud_data(df_raw)
target_col = "class"

print(f"Data shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"Data types:\\n{df.dtypes}")
print()

### Target Separation

In [ ]:
print("2. Separating features and target...")
X, y = separate_features_target(df, target_col)
print(f"Features shape: {X.shape}, Target shape: {y.shape}")

# Display target distribution
fraud_percentage = y.mean() * 100
print(f"Fraud rate: {fraud_percentage:.4f}%")
print(f"Class distribution:\\n{y.value_counts().to_dict()}")

- The target variable `class` indicates whether a transaction is fraudulent (1)
or legitimate (0).
- The dataset is highly imbalanced, with fraudulent transactions representing
a very small proportion of all samples. This motivates the use of imbalance-aware
metrics and resampling techniques.

### Stratified Train-Test Split

In [ ]:
# Stratified Train-Test Split
print("\n3. Creating stratified train-test split...")
X_train, X_test, y_train, y_test = stratified_split(X, y)
print(f"Training set: {X_train.shape}, Test set: {X_test.shape}")
print(f"Training fraud rate: {y_train.mean()*100:.4f}%")
print(f"Test fraud rate: {y_test.mean()*100:.4f}%")

- A stratified split preserves the fraud ratio across training and test sets.

## PREPROCESSING PIPELINE

In [ ]:
# Preprocessing Pipeline
print("\n4. Applying preprocessing pipeline...")
preprocessor = build_preprocessor(X)
X_train_p = preprocessor.fit_transform(X_train)
X_test_p = preprocessor.transform(X_test)
print(f"Preprocessed training features shape: {X_train_p.shape}")
print(f"Preprocessed test features shape: {X_test_p.shape}")

### HANDLE CLASS IMBALANCE (SMOTE)

In [ ]:
print("\n5. Handling class imbalance with SMOTE...")
X_train_res, y_train_res = apply_smote(X_train_p, y_train)
print(f"Resampled training set shape: {X_train_res.shape}, {y_train_res.shape}")


In [ ]:
# Visualize class distribution before/after SMOTE
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

sns.countplot(x=y_train, ax=axes[0])
axes[0].set_title("Before SMOTE (Training Set)")
axes[0].set_ylabel("Count")

sns.countplot(x=y_train_res, ax=axes[1])
axes[1].set_title("After SMOTE (Training Set)")
axes[1].set_ylabel("Count")

plt.tight_layout()
# Save figure
save_figures("../reports/class_distribution_smote.png", dpi=150, bbox_inches="tight")
plt.savefig("../reports/class_distribution_smote.png", dpi=150, bbox_inches='tight')

plt.show()

print("\nSMOTE is applied only to the training data to prevent information leakage.")

SMOTE is applied **only to the training data** to prevent information leakage.

## Model Training and Evaluation
### Train and Evaluate Baseline Model: Logistic Regression

In [ ]:
# Train and Evaluate Baseline Model: Logistic Regression
print("\n" + "="*80)
print("6. Training and evaluating Logistic Regression (baseline)...")
print("="*80)

lr = logistic_regression()
lr.fit(X_train_res, y_train_res)
lr_metrics = evaluate_model(lr, X_test_p, y_test)

print("Logistic Regression Metrics:")
print(f"F1-Score: {lr_metrics['f1']:.4f}, AUC-PR: {lr_metrics['auc_pr']:.4f}")
print(f"Precision: {lr_metrics['precision']:.4f}, Recall: {lr_metrics['recall']:.4f}")

# Confusion matrix
plot_confusion_matrix(lr_metrics['confusion_matrix'], "Logistic Regression")
plt.savefig("../reports/confusion_matrix_logistic_regression.png", dpi=150, bbox_inches='tight')
plt.show()



- The Logistic Regression confusion matrix shows strong performance on legitimate
transactions but limited recall for fraudulent cases. This indicates that while
the model is conservative and interpretable, it misses a portion of fraud due
to its linear decision boundary in a highly imbalanced setting.

### Train and Evaluate Ensemble Model: Random Forest

In [ ]:
# Train and Evaluate Random Forest (with threshold optimization)
print("\n" + "="*80)
print("7. Training and evaluating Random Forest with threshold optimization...")
print("="*80)

# Create threshold-optimized Random Forest
optimal_threshold = 0.65  # Based on previous analysis
rf_optimized = threshold_optimized_random_forest(
    n_estimators=200, 
    max_depth=None, 
    threshold=optimal_threshold
)
rf_optimized.fit(X_train_res, y_train_res)

# Evaluate with optimal threshold
rf_metrics_optimal = evaluate_model(rf_optimized.model, X_test_p, y_test, threshold=optimal_threshold)

print(f"Random Forest with threshold {optimal_threshold}:")
print(f"F1-Score: {rf_metrics_optimal['f1']:.4f}, AUC-PR: {rf_metrics_optimal['auc_pr']:.4f}")
print(f"Precision: {rf_metrics_optimal['precision']:.4f}, Recall: {rf_metrics_optimal['recall']:.4f}")

# Confusion matrix at optimal threshold
plot_confusion_matrix(rf_metrics_optimal['confusion_matrix'], f"Random Forest (Threshold={optimal_threshold})")
plt.savefig(f"../reports/confusion_matrix_rf_threshold_{optimal_threshold}.png", dpi=150, bbox_inches='tight')
plt.show()

- The Random Forest confusion matrix demonstrates improved detection of fraudulent
transactions, with fewer false negatives compared to Logistic Regression. This
reflects the model’s ability to capture complex, non-linear relationships in
transaction behavior, which is critical for fraud detection.

In [ ]:
# 8️⃣ Threshold Analysis
print("\n" + "="*80)
print("8. Performing comprehensive threshold analysis...")
print("="*80)

# Get probabilities for threshold analysis
y_proba = rf_optimized.model.predict_proba(X_test_p)[:, 1]

# Evaluate at multiple thresholds
print("\nEvaluating Random Forest at different thresholds:")
thresholds_full = [0.1, 0.2, 0.3, 0.4, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8]
threshold_results = evaluate_model_at_thresholds(rf_optimized.model, X_test_p, y_test, thresholds=thresholds_full)

# Display threshold analysis results
print("\nThreshold Analysis Results:")
print(threshold_results[['threshold', 'f1', 'precision', 'recall', 'accuracy']].round(3))

# Find optimal threshold programmatically
optimal_thresh_calculated, best_f1, all_results = find_optimal_threshold(
    rf_optimized.model, X_test_p, y_test, metric='f1'
)
print(f"\\nOptimal threshold (maximizing F1): {optimal_thresh_calculated:.3f}")
print(f"Best F1 score: {best_f1:.3f}")

# Visualize threshold analysis
plot_threshold_analysis(threshold_results)
plt.savefig("../reports/threshold_analysis.png", dpi=150, bbox_inches='tight')

plot_metric_tradeoff(threshold_results)
plt.savefig("../reports/metric_tradeoff.png", dpi=150, bbox_inches='tight')

In [ ]:
# 9️⃣ Business Metrics Analysis
print("\n" + "="*80)
print("9. Calculating business metrics...")
print("="*80)

# Define business costs (customize based on your business context)
COST_FALSE_POSITIVE = 10   # Cost of investigating a false alarm
COST_FALSE_NEGATIVE = 100  # Cost of missing a fraud

print(f"Business cost assumptions:")
print(f"  Cost of false positive (investigation): ${COST_FALSE_POSITIVE}")
print(f"  Cost of false negative (missed fraud): ${COST_FALSE_NEGATIVE}")

# Calculate business metrics at different thresholds
print("\nBusiness metrics at key thresholds:")
key_thresholds = [0.5, optimal_thresh_calculated, 0.7]
for thresh in key_thresholds:
    y_pred_thresh = (y_proba >= thresh).astype(int)
    biz_metrics = get_business_metrics(
        y_test, y_pred_thresh, 
        cost_fp=COST_FALSE_POSITIVE, cost_fn=COST_FALSE_NEGATIVE
    )
    
    print(f"\\nThreshold {thresh:.2f}:")
    print(f"  Total cost: ${biz_metrics['total_cost']:,.0f}")
    print(f"  Cost per transaction: ${biz_metrics['cost_per_transaction']:.2f}")
    print(f"  Fraud capture rate: {biz_metrics['fraud_capture_rate']*100:.1f}%")
    print(f"  False alarm rate: {biz_metrics['false_alarm_rate']*100:.1f}%")
    print(f"  False positives: {biz_metrics['false_positives']:,}")
    print(f"  False negatives: {biz_metrics['false_negatives']:,}")

## Precision-Recall curves

In [ ]:
# 🔟 Precision-Recall Curves Comparison
print("\n" + "="*80)
print("10. Comparing Precision-Recall curves...")
print("="*80)

plt.figure(figsize=(8, 6))
plot_pr_curve(y_test, lr_metrics["y_prob"], "Logistic Regression")
plot_pr_curve(y_test, y_proba, f"Random Forest (AUC-PR = {rf_metrics_optimal['auc_pr']:.3f})")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve Comparison")
handles, labels = plt.gca().get_legend_handles_labels()
if handles:
    plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("../reports/precision_recall_curves.png", dpi=150, bbox_inches='tight')
plt.show()



Precision–Recall curves are especially informative for imbalanced datasets such
as fraud detection. The Random Forest model maintains higher precision at higher
recall levels compared to Logistic Regression, confirming its superior ability
to identify fraudulent transactions while controlling false positives.

## STRATIFIED K-FOLD CV (k=5)

In [ ]:
# Cross-Validation with Threshold Optimization
print("\n" + "="*80)
print("11. Cross-validation with threshold optimization...")
print("="*80)

# Use subset for faster CV
# subset_size = min(20000, len(X_train_p))
subset_size = min(20000, X_train_p.shape[0])
print(f"Using subset of {subset_size} samples for CV...")

X_cv = X_train_p[:subset_size]
y_cv = y_train.iloc[:subset_size].to_numpy()

# Perform CV with threshold optimization
cv_results = stratified_cv_with_threshold_opt(
    rf_optimized.model, 
    X_cv, 
    y_cv, 
    n_splits=5
)

print(f"Cross-Validation Results:")
print(f"  Optimal threshold (CV): {cv_results['best_threshold']:.3f}")
print(f"  Best F1 score (CV): {cv_results['best_f1_mean']:.3f} ± {cv_results['best_f1_std']:.3f}")

# Display threshold performance summary
print("\nThreshold performance across CV folds:")
print(cv_results['threshold_performance'].head(10).round(3))


In [ ]:
from src.model_tuning import tune_random_forest
from src.cv import stratified_cv

# Tune Random Forest on training data
best_rf, best_params = tune_random_forest(X_train_p, y_train)

print("Best Random Forest parameters:")
print(best_params)

# Cross-validate the tuned model
cv_results_rf = stratified_cv(best_rf, X_train_p, y_train)

print("Tuned Random Forest CV Results:")
print(cv_results_rf)


In [ ]:
# Final Model Selection and Training
print("\n" + "="*80)
print("12. Training final model with optimal threshold...")
print("="*80)

# Determine final threshold (choose based on business needs)
FINAL_THRESHOLD = optimal_thresh_calculated  # or use cv_results['best_threshold']
print(f"Selected final threshold: {FINAL_THRESHOLD:.3f}")

# Train final model on full training data
print("Training final Random Forest on full dataset...")
rf_final = threshold_optimized_random_forest(
    n_estimators=200,
    max_depth=None,
    threshold=FINAL_THRESHOLD
)
rf_final.fit(X_train_res, y_train_res)

# Evaluate final model
final_metrics = evaluate_model(rf_final.model, X_test_p, y_test, threshold=FINAL_THRESHOLD)

print(f"\\nFinal Random Forest Metrics (Threshold={FINAL_THRESHOLD:.3f}):")
print(f"F1-Score: {final_metrics['f1']:.4f}")
print(f"AUC-PR: {final_metrics['auc_pr']:.4f}")
print(f"Precision: {final_metrics['precision']:.4f}")
print(f"Recall: {final_metrics['recall']:.4f}")

# Final confusion matrix
plot_confusion_matrix(final_metrics["confusion_matrix"], f"Final Random Forest (Threshold={FINAL_THRESHOLD:.2f})")
plt.savefig("../reports/confusion_matrix_final_model.png", dpi=150, bbox_inches='tight')


In [ ]:
# Model Comparison Table
print("\n" + "="*80)
print("13. Model comparison...")
print("="*80)

# Also get RF metrics at default threshold for comparison
rf_metrics_default = evaluate_model(rf_optimized.model, X_test_p, y_test, threshold=0.5)

model_comparison = pd.DataFrame({
    "Model": [
        "Logistic Regression (default)", 
        "Random Forest (threshold=0.5)", 
        f"Random Forest (threshold={FINAL_THRESHOLD:.2f})"
    ],
    "F1-Score": [
        lr_metrics["f1"], 
        rf_metrics_default["f1"], 
        final_metrics["f1"]
    ],
    "AUC-PR": [
        lr_metrics["auc_pr"], 
        rf_metrics_default["auc_pr"], 
        final_metrics["auc_pr"]
    ],
    "Precision": [
        lr_metrics["precision"], 
        rf_metrics_default["precision"], 
        final_metrics["precision"]
    ],
    "Recall": [
        lr_metrics["recall"], 
        rf_metrics_default["recall"], 
        final_metrics["recall"]
    ]
})

print("\nModel Comparison:")
print(model_comparison.to_string(index=False))

# Calculate improvement
improvement = ((final_metrics["f1"] - rf_metrics_default["f1"]) / rf_metrics_default["f1"]) * 100
print(f"\\nF1-score improvement with threshold optimization: {improvement:.1f}%")


In [ ]:

# Save Final Model and Metadata
print("\n" + "="*80)
print("14. Saving final model and metadata...")
print("="*80)

# Save the model
model_path = "../models/final_fraud_model.pkl"
joblib.dump({
    'model': rf_final.model,
    'preprocessor': preprocessor,
    'threshold': FINAL_THRESHOLD,
    'metrics': final_metrics
}, model_path)
print(f"Model saved to: {model_path}")

# Save threshold optimizer
optimizer_path = "../models/threshold_optimizer.pkl"
joblib.dump(rf_final, optimizer_path)
print(f"Threshold optimizer saved to: {optimizer_path}")

# Save evaluation results
results_path = "../reports/model_evaluation_results.csv"
model_comparison.to_csv(results_path, index=False)
print(f"Evaluation results saved to: {results_path}")

# Save threshold analysis results
threshold_results_path = "../reports/threshold_analysis_results.csv"
threshold_results.to_csv(threshold_results_path, index=False)
print(f"Threshold analysis results saved to: {threshold_results_path}")


In [ ]:

# Summary and Recommendations
print("\n" + "="*80)
print("FINAL SUMMARY AND RECOMMENDATIONS")
print("="*80)

print(f"""
1. MODEL PERFORMANCE:
   - Best model: Random Forest with threshold = {FINAL_THRESHOLD:.3f}
   - F1-Score: {final_metrics['f1']:.3f} (vs {rf_metrics_default['f1']:.3f} at default threshold 0.5)
   - Improvement: {improvement:.1f}%
   - Precision: {final_metrics['precision']:.3f}
   - Recall: {final_metrics['recall']:.3f}

2. BUSINESS IMPLICATIONS:
   - Fraud capture rate: {final_metrics['recall']*100:.1f}% of frauds detected
   - Investigation efficiency: {final_metrics['precision']*100:.1f}% of flagged cases are actual fraud

3. DEPLOYMENT RECOMMENDATIONS:
   - Use threshold {FINAL_THRESHOLD:.3f} for production
   - Consider tiered approach:
     * Score >= 0.85: Auto-block (high confidence)
     * Score >= {FINAL_THRESHOLD:.2f}: Human review
     * Score < {FINAL_THRESHOLD:.2f}: Auto-allow

4. THRESHOLD OPTIMIZATION BENEFITS:
   - Default threshold (0.5) was suboptimal for imbalanced fraud detection
   - Threshold optimization improved F1 by {improvement:.1f}%
   - Better balance between catching fraud and minimizing false alarms
""")

print("\n" + "="*80)
print("MODELING COMPLETE")
print("="*80)
    


## Final Model Selection

- AUC-PR is used instead of ROC-AUC because it is more informative for highly
imbalanced datasets such as fraud detection.

- The Random Forest model outperformed Logistic Regression in both F1-Score
and AUC-PR, indicating a stronger ability to detect fraudulent transactions
while minimizing false negatives.

- Although Logistic Regression offers higher interpretability, its linear
decision boundary limits performance on complex fraud patterns.
Random Forest captures non-linear interactions between behavioral, temporal,
and geolocation features.

- Given the business importance of fraud detection accuracy and the observed
performance gains, Random Forest was selected as the final model.